# Chapter 5: Support Vector Machines

<i>Support Vector Machines</i> are a machine learning algorithm capable of both
linear and nonlinear classification. They are best for small- to medium-sized
datasets.

## Linear SVM Classification

One useful property of SVMs is that unlike other linear classifiers, SVMs'
decision boundary will be as far from the training instances as possible. This
is called <i>large margin classification.</i> SVMs try to find the largest possible margin (or "street") that can be used as a decision boundary between classes.

Adding training instances far off the street will not affect the outcome
of training the model. Instances on the edge of the street are referred to as <i>support vectors.</i>

SVMs are also sensitive to the scale of the data being used to train the model.

### Soft Margin Classification

<i>Hard margin classification</i> is when a model requires each instance of each class be on the same side of the street. Training models this way is only possible when the data is linearly separable and is sensitive to outliers.

Another way to train the model is to aim for the widest street possible while limiting the number of <i>margin violations</i> (instances that end up on the street or on the wrong side). This method is called <i>soft margin classification</i>. In Scikit-Learn, you can tune how lenient the model is with margin violations using the $C$ hyperparameter. Large values of $C$ allow for fewer margin violations and a thinner street, whereas smaller values of $C$ allow for more margin violations and a larger street.

In [6]:
# The following code uses Scikit-Learn to classify the iris dataset using
# an SVM

import warnings
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

warnings.filterwarnings("ignore")

iris = datasets.load_iris()
X = iris['data'][:, (2, 3)]  # petal length, width
y = (iris['target'] == 2).astype(np.float64)

svm_clf = Pipeline([
  ('scaler', StandardScaler()),
  ('linear_svc', LinearSVC(C=1, loss='hinge'))
])

svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [7]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

You can also uses the `SVC` class using `SVC(kernel='linear', C='1')`, but it is much slower than `LinearSVC`. You can also use the `SGDClassifier` class using `SGDClassifier(loss='hinge', alpha=1/(m*C))` which would apply Stochastic Gradient Descent to train a linear SVM classifier.

## Nonlinear SVM Classification

Often datasets are not linearly separable. One thing you can do is add polynomial features to the dataset to make it linearly separable. An example of this is shown below.

In [8]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons()

poly_svm_clf = Pipeline([
  ('poly_features', PolynomialFeatures(degree=3)),
  ('scaler', StandardScaler()),
  ('svm_clf', LinearSVC(C=10, loss='hinge')),
])

poly_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('poly_features', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

### Polynomial Kernel

It can be difficult to choose the right degree of polynomial to use for transforming the data before fitting. SVMs are able train themselves to use
polynomials without you having to add them using the <i>kernel trick</i>. An example of doing this with Scikit-Learn is below using polynomial features up to degree 3.

In [9]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
  ('scaler', StandardScaler()),
  ('svm_clf', SVC(kernel='poly', degree=3, coef0=0.1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=5, cache_size=200, class_weight=None, coef0=0.1,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='poly', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

If the model is underfitting, you can increase the degree (or lower the degree if it is overfitting). You can also tune the `coef0` hyperparameter which controls how sensitive the model is to high degree polynomials versus low degree polynomials.

### Adding Similarity Features

Another way to solve nonlinear problems is to add features computed using a <i>similarity function</i> that measures how much each instance resembles a <i>landmark</i>. One example of a similarity function is called the Gaussian <i>Radial Basis Function</i> (RBF),

$$ \phi_\gamma(\mathbf{x}, \mathbf{\ell}) =
\exp\left(-\gamma\,||\mathbf{x} - \mathbf{\ell} ||^2\right) $$

which has a range of 0 (very far from the landmark) to 1 (at the landmark). Often one uses each instance in the training set as a landmark, then drops the original features. However, for large datasets this can greatly increase the time it takes to train a model.

### Guassian RBF Kernel

Scikit-Learn's `SVC` class can use a kernel trick to train a model as if it had computed all of the similarity features without the combinatorial overhead. An example implementation is below.

In [10]:
rbf_kernel_clf = Pipeline([
  ('scaler', StandardScaler()),
  ('', SVC(kernel='rbf', C=.001, gamma=5)),
])

rbf_kernel_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('', SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

the hyerperameter parameter `gamma` influences the shape of the bell curve. See the mathematical definition for the Gaussian RBF above.

### Computational Complexity

The `LinearSVC` class uses a library called [`liblinear`](https://www.csie.ntu.edu.tw/~cjlin/liblinear/), which uses an [optimized algorithm](https://www.csie.ntu.edu.tw/~cjlin/papers/cddual.pdf) for training linear kernels. It does not support the kernel trick. The computational complexity is $O(n \times m)$ where $n$ is the dimension of the vectors in the training set and $m$ is the size of the training set.

The `SVC` class uses the [`libsvm`](https://www.csie.ntu.edu.tw/~cjlin/libsvm/) which implements an [algorithm](https://www.microsoft.com/en-us/research/publication/sequential-minimal-optimization-a-fast-algorithm-for-training-support-vector-machines/?from=http%3A%2F%2Fresearch.microsoft.com%2Fpubs%2F69644%2Ftr-98-14.pdf) which supports the kernel trick. The training time complexity of training an algorithm is generally in the range of $O(m^2 \times n)$ to $O(m^3 \times n)$, making it very slow when the training set has a large size. This makes nonlinear SVM classifiers best for small to medium size, complex training sets.

## SVM Regression

SVMs also support regression by reversing the objective. It tries to create streets which intersects the most training instances. Scikit-Learn's `LinearSVR` class implements this regression, and uses the hyperparameter $\epsilon$ to determine the width of the margin, the larger $\epsilon$, the larger the margin used when determining the optimal "street" to train the model. Below is an example of using Scikit-Learn's `LinearSVR` class.

In [12]:
from sklearn.svm import LinearSVR

svm_reg = Pipeline([
  ('scaler', StandardScaler()),
  ('svr', LinearSVR(epsilon=1.5)),
])
svm_reg.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0))])

For nonlinear regression, the `SVR` class can fit the data with a polynomial function. It can be tuned with the $\epsilon$ hyperparameter, also a regularization parameter $C$, and you can set the maximum degree the algorithm will use. Below is an example of Scikit-Learn's SVR Class.

In [11]:
from sklearn.svm import SVR

svm_poly_reg = Pipeline([
  ('scaler', StandardScaler()),
  ('svr', SVR(kernel='poly', degree=2, C=100, epsilon=0.1)),
])
svm_poly_reg.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1,
  gamma='auto_deprecated', kernel='poly', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))])

## Under the Hood

Below is notes on the mathematics behind the implementation of SVMs. Unlike chapter 4, which used the vector $\theta$ as the values being computed by the training algorithm, in this section we will use $b$ to denote the bias term and $\mathbf{w}$ as the weights vector.

### Decision Function and Predictions

The linear SVM classifier predicts the class of an instance by first computing the decision function

$$ \mathbf{w}^T \cdot \mathbf{x} + b
= \left( \sum\limits_{i = 1}^n w_nx_n \right) + b. $$

The classifier then makes the decision based on the following criteria

$$ \hat{y} = \left\{ \begin{matrix}
0 && \text{if} \;\; \mathbf{w}^T \cdot \mathbf{x} + b < 0 \\
1 && \text{if} \;\; \mathbf{w}^T \cdot \mathbf{x} + b \geq 0
\end{matrix} \right. $$

The decision boundary is the set of all points in the training set space where the decision function is equal to 0. The edges of the margin are where the decision function is equal to $\pm1$.

### Training Objective

The slope of the decision function is equal to the norm of the weight vector, $||\mathbf{w}||$. This means that the width of the margin is inversely proportional to the norm of $\mathbf{w}$. If we want to avoid any margin violations, the decision function must be greater than 1 for positive instances and less than -1 for negative instances. This constraint can be expressed as

$$ t^{(i)}\left( \mathbf{w}^T \cdot \mathbf{x}^{(i)} + b \right) \geq 1 $$

where $t^{(i)}$ is -1 for all negative instances and 1 for all positive instances. Thus the hard margin linear SVM classifier objective is the <i>constrainted optimization</i> below

$$ \underset{\mathbf{w},\;b}{\text{minimize}} \;\; \frac{1}{2}
\mathbf{w}^T \cdot \mathbf{w} $$

subject to the constraint that

$$ t^{(i)} \left( \mathbf{w}^T \cdot \mathbf{x}^{(i)} + b \right) \geq 1. $$

To get the soft margin objective, wee need to introduce a <i>slack variable</i>, $\zeta^{(i)} \geq 0$ for each training instance. Each $\zeta^{(i)}$ measures how much the $i^\text{th}$ instance is allowed to violate the margin. Therefore for soft margin classification, we have two problems: minimize the slack variables as well as $\frac{1}{2}\mathbf{w}^T\cdot\mathbf{x}$. The $C$ hyperparameter allows us to define the trade-off between the two objectives. The constrained optimization problem for soft margin classification is given by

$$ \underset{\mathbf{w},\;b,\;\zeta}{\text{minimize}} \;\;
\frac{1}{2}\mathbf{w}^T\cdot\mathbf{w} \, + \,
C\sum\limits_{i=1}^m \zeta^{(i)} $$

subject to the constraint that

$$ t^{(i)} \left( \mathbf{w}^T \cdot \mathbf{x}^{(i)} + b \right)
\geq 1 - \zeta^{(i)} \;\; \text{and} \;\; \zeta^{(i)} \geq 0 \;\; 
\text{for} \;\; i = 1, 2, ..., m. $$

### Quadratic Programming

The hard and soft margin classification problems belong to a class of problems known as [<i>Quadratic Programming</i>](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf) (QP) problems. The general problem formulation for QP problems is given by

$$ \underset{\mathbf{p}}{\text{minimize}} \;\;
\frac{1}{2} \mathbf{p}^T \cdot \mathbf{H} \cdot \mathbf{p} +
\mathbf{f}^T \cdot \mathbf{p} $$

subject to

$$ \mathbf{A} \cdot \mathbf{p} \leq \mathbf{b} $$

where

- $\mathbf{p}$ is an $n_p$-dimensional vector ($n_p$ is the number of parameters)
- $\mathbf{H}$ is an $n_p \times n_p$ matrix
- $\mathbf{f}$ is an $n_p$-dimensional vector
- $\mathbf{A}$ is an $n_c \times n_p$ matrix ($n_c$ is the number of constraints)
- $\mathbf{b}$ is an $n_c$-dimensional vector

QP problems can be solved using off-the-shelf solvers which are available in different code libraries.

### The Dual Problem

Given a constrained optimization problem, i.e. the <i>primal problem</i>, there is a different but related problem known as its <i>dual problem</i>. In the special case that the objective function is convex, and the inequality constraint is both convex and continuously differentiable (such as SVMs), the solution to the dual problem is also a solution to the primal problem. The dual form of the SVM problem is given by

$$ \underset{\alpha}{\text{minimize}} \;\;
\frac{1}{2} \sum\limits_{i=1}^m \sum\limits_{j=1}^m
\alpha^{(i)} \alpha^{(j)} t^{(i)} t^{(j)} \mathbf{x}^{(i)\,T}
\cdot \mathbf{x}^{(j)} \; - \; \sum\limits_{i=1}^m \alpha^{(i)} $$

subject to the constraint that

$$ \alpha^{(i)} \geq 0 \;\; \text{for} \;\; i=1,2,...,m. $$

Once you find the $\hat{\alpha}$ which minimizes the objective function, you can compute $\hat{\mathbf{w}}$ and $\hat{b}$ that minimize the primal problem using

$$ \hat{\mathbf{w}} = \sum\limits_{i=1}^m \hat{\alpha}^{(i)} t^{(i)}
\mathbf{x}^{(i)} \\
\hat{b} = \frac{1}{n_s} \underset{\hat{\alpha}^{(i)} > 0}{\sum\limits_{i=1}^m}
\left( t^{(i)} - \hat{\mathbf{w}}^T \cdot \mathbf{x}^{(i)} \right). $$

The dual problem is generally faster to solve than the primal one, and it also it is possible to use the kernel trick on the dual problem.

### Kernelized SVM

Suppose you want to apply a $2^\text{nd}$-degree polynomial transformation to a two-dimensional dataset using a function, $\phi$, the transformation would be given by

$$ \phi(\mathbf{x}) = \phi\left(\left[ \begin{matrix}
x_1 \\ x_2
\end{matrix} \right]\right) =
\left[ \begin{matrix}
x_1^{\;2} \\ \sqrt{2} \, x_1x_2 \\ x_2^{\;2}
\end{matrix} \right]. $$

If you were to take the dot product after transforming two vectors, $\mathbf{a}$ and $\mathbf{b}$, you will find that

$$ \phi(\mathbf{a})^T \cdot \phi(\mathbf{b}) =
\left[ \begin{matrix} a_1^{\;2} \\ \sqrt{2}\,a_1a_2 \\ a_2^{\;2} \end{matrix}
\right]^{\;T}
\cdot \left[ \begin{matrix}
b_1^{\;2} \\ \sqrt{2}\,b_1b_2 \\ b_2^{\;2}
\end{matrix} \right] = a_1^{\;2}b_1^{\;2} + 2 \, a_1b_1a_2b_2 +
a_2^{\;2}b_2^{\;2} = \left( \mathbf{a}^T \cdot \mathbf{b} \right)^2. $$

This means that we can apply the $2^\text{nd}$ degree polynomial transformation of the dot product in the dual SVM problem by simply squaring the quantity $\mathbf{x}^T \cdot \mathbf{x}$. This is how the polynomial SVM kernel is able to apply the polynomial transformation during training with much more computational efficiency.

The function $K(\mathbf{a}, \mathbf{b}) = \left( \mathbf{a}^T \cdot \mathbf{b} \right)^2$ is called the $2^\text{nd}$-degree <i>polynomial kernel</i> where a <i>kernel</i> is a function capable of computing the quantity

$$ \phi(\mathbf{a})^T \cdot \phi(\mathbf{b}) $$

without needing to compute either $\phi(\mathbf{a})$ or $\phi(\mathbf{b})$. Some of the most commonly used kernels are

$$ \begin{matrix}
\text{Linear:} && K(\mathbf{a},\mathbf{b}) = \mathbf{a}^T \cdot \mathbf{b} \\
\text{Polynomial:} && K(\mathbf{a},\mathbf{b}) =
\left( \gamma\,\mathbf{a}^T \cdot \mathbf{b} + r \right)^d \\
\text{Gaussian RBF:} && K(\mathbf{a},\mathbf{b}) =
\exp\left( -\gamma || \mathbf{a} - \mathbf{b} ||^2 \right) \\
\text{Sigmoid:} && K(\mathbf{a},\mathbf{b}) =
\tanh\left( \gamma\,\mathbf{a}^T \cdot \mathbf{b} + r \right)
\end{matrix} $$

According to <i>Mercer's theorem</i>, if a function $K(\mathbf{a},\mathbf{b})$ meets a set of conditions called <i>Mercer's conditions</i>, then there exists a function $\phi$ that maps $\mathbf{a}$ and $\mathbf{b}$ into another space (possibly with a higher dimension) such that $K(\mathbf{a},\mathbf{b}) = \phi(\mathbf{a})^T \cdot \phi(\mathbf{b})$, so you can use $K$ as a kernel even if you do not know what $\phi$ is. Some kernels (e.g. the Sigmoid kernel) do not follow all of Mercer's condition but still work in practice.

The caveat is that the formula which determines the weights vector and bias term from the solution to the dual problem require that you compute $\phi\left(\mathbf{x}^{(i)}\right)$ and that $\hat{\mathbf{w}}$ has the same dimension as $\phi\left(\mathbf{x}^{(i)}\right)$ which can be very large or even infinite. However, it is possible to express the decision function using the kernel:

$$ h_{\hat{\mathbf{w}},\,\hat{b}} \left( \phi\left( \mathbf{x}^{(n)} \right) \right) = \hat{\mathbf{w}}^T \cdot \phi\left( \mathbf{x}^{(n)} \right) + \hat{b} = \left( \sum\limits_{i=1}^m \hat{\alpha}^{(i)} t^{(i)} \phi\left( \mathbf{x}^{(i)} \right) \right)^T \cdot \phi\left( \mathbf{x}^{(n)} \right) + \hat{b} \\
= \sum\limits_{i=1}^m \hat{\alpha}^{(i)} t^{(i)} \left( \phi\left( \mathbf{x}^{(i)} \right)^T \cdot \phi\left( \mathbf{x}^{(n)} \right) \right) + \hat{b} \\
= \underset{\hat{\alpha}^{(i)} > 0}{\sum\limits_{i=1}^m} \hat{\alpha}^{(i)} t^{(i)} K\left( \mathbf{x}^{(i)}, \mathbf{x}^{(n)} \right) + \hat{b}. $$

The terms where $\hat{\alpha}^{(i)}$ are greater than 0 are the <i>support vectors</i>, influencing their position can influence the decision boundary. This formula shows that in order to get a prediction, we need only compute the kernel for the training instances which are support vectors. The bias term $\hat{b}$ is given by the formula below

$$ \hat{b} = \frac{1}{n_s} \sum\limits_{i=1}^m \left( 1 - t^{(i)}\hat{\mathbf{w}}^T \cdot \phi\left( \mathbf{x}^{(i)} \right) \right) =
\frac{1}{n_s} \sum\limits_{i=1}^m \left(
1 - t^{(i)} \left( \sum\limits_{j=1}^m \hat{\alpha}^{(j)} t^{(j)} \phi\left( \mathbf{x}^{(j)} \right) \right)^T \cdot \phi\left( \mathbf{x}^{(i)} \right)
\right) \\
= \frac{1}{n_s} \underset{\hat{\alpha}^{(i)} > 0}{\sum\limits_{i=1}^m}
\left( 1 - t^{(i)} \underset{\hat{\alpha}^{(j)} > 0}{\sum\limits_{j=1}^m}
\hat{\alpha}^{(j)}t^{(j)}K\left( \mathbf{x}^{(i)}, \mathbf{x}^{(j)} \right)
\right) $$

again, the model needs only to compute the kernel for the support vectors, and not every training instance, saving a lot of computational effort.

### Online SVMs

One way to have an online linear SVM is to have it use gradient descent with the following cost function

$$ J(\mathbf{w}, b) = \frac{1}{2} \, \mathbf{w}^T \cdot \mathbf{w} \; + \;
C \sum\limits_{i=1}^n max\left(0, 1 - t^{(i)} \left( \mathbf{w}^T \cdot \mathbf{x}^{(i)} + b \right) \right)
$$

the first term in the sum keeps the norm of $\mathbf{w}$ minimized, the second term computes the total number of margin violations. The downside is that this method of training converges much more slowly than use a QP solver to find the solution to the dual problem.

There exists online kernelized SVMs, for example using ["Incremental and Decremental SVM Learning"](http://isn.ucsd.edu/papers/nips00_inc.pdf) or ["Fast Kernel Classifiers with Online and Active Learning,"](http://www.jmlr.org/papers/volume6/bordes05a/bordes05a.pdf) but it is probably better to use neural networks for online nonlinear problems.

### Hinge Loss

The function $max(0, 1 - t)$ is called the <i>hinge loss</i> function. The derivative is equal to -1 when $t$ is in $(-\infty,1)$, the function is not differentiable when $t=1$ and is 0 elsewhere. Even though it is not differentiable at $t=1$, you can use the [<i>subderivative</i>](https://en.wikipedia.org/wiki/Subderivative) to use this function in Gradient Descent (like using the subgradient for Lasso regression).

## Exercises

### 1. What is the fundamental idea behind Support Vector Machines?

SVM classifiers try to find a decision boundary with a margin where there are as few training instances in the margin around the decision boundary. SVM regressors, on the other hand, try to find margins around the decision function which has as many training instances contained in them.

### 2. What is a support vector?

A support vector is a vector that is close to the decision boundary. Changing the features of these vectors slightly is likely to change the resulting decision boundary.

### 3. Why is it important to scale the inputs when using SVMs?

Scaling the feature set prevents one particular feature's value overpowering and information that may also be in features with a smaller value.

### 4. Could an SVM output a confidence score when it classifies an instance? What about a probability?

You could use the distance from the function's decision boundary as a measure of confidence in the model's decision. It will be less confident when a new instance is close to the decision boundary, so the function should be monotonically increasing as a function of the sample's distance from the decision boundary.

You could also use a logistic curve which is a function of the new instance's distance from the deicision boundary which asymptotically approaches 0 on one side and 1 on the other. This function would give you the probability that the model belongs to a particular class.

### 5. Should you use the primal or dual form of the SVM problem to train a model on a training set with millions of instances with hundreds of features?

You should use the primal problem and train the model with a QP solver. You will need to transform nonlinear data since the primal problem is not capable of the kernel trick. The primal problem is faster when the number of training instances is largely greater than the number of features.

### 6. Say you trained an SVM classifier with an RBF kernel. It seems to underfit the training set: should you increase or decrease $\gamma$?

You should decrease the $\gamma$ hyperparameter. Doing so will increase the influence the instances in the training set has over the decision boundary.

### 7. How should you set the QP parameters, $( \mathbf{H}, \mathbf{f}, \mathbf{A},$ and $\mathbf{b})$ to solve the soft margin linear SVM classifier problem using an off-the-shelf QP solver?

It was stated above that the general formula for a QP problem is given by

$$ \underset{\mathbf{p}}{\text{minimize}} \;\;
\frac{1}{2} \mathbf{p}^T \cdot \mathbf{H} \cdot \mathbf{p} +
\mathbf{f}^T \cdot \mathbf{p} $$

and is subjected to the constraint that

$$ \mathbf{A} \cdot \mathbf{p} \leq \mathbf{b} $$

Also, it was stated that dual form of the linear soft margin SVM problem is given by

$$ \underset{\alpha}{\text{minimize}} \;\;
\frac{1}{2} \sum\limits_{i=1}^m \sum\limits_{j=1}^m
\alpha^{(i)} \alpha^{(j)} t^{(i)} t^{(j)} \mathbf{x}^{(i)\,T}
\cdot \mathbf{x}^{(j)} \; - \; \sum\limits_{i=1}^m \alpha^{(i)} $$

and is subject to the constraint that

$$ \alpha^{(i)} \geq 0 \;\; \text{for} \;\; i=1,2,...,m. $$

It follows that the paramters that should be used for an off-the-shelf QP solver, substitutiong $\alpha$ for the term $\mathbf{p}$, the should be:

$$ \mathbf{H} = ||h_{ij}|| \;\;
\text{for} \;\; i,j = 1, 2, ..., m $$

where each term in the matrix $h_{ij}$ is given by

$$ h_{ij} = t^{(i)}t^{(j)} (\mathbf{x}^{(i)\,T} \cdot \mathbf{x}^{(j)}). $$

The term $\mathbf{f}$ should just be an $m$-dimensional vector where each component is -1. Finally, the term $\mathbf{A}$ should simply be the $m\times m$ identity matrix and the term $\mathbf{b}$ should just be the $m$-dimensional zero vector.

### 8. Train a `LinearSVC` classifier on a linearly separable dataset. Then train an `SGDClassifier` on the same dataset. See if you can get them to produce roughly the same model.

For this exercise I will use the Iris dataset tot try to find the species of the flower using petal width and petal length. First, we download the data.

In [0]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:, (2,3)]
y = iris.target

Next we see that using the `LinearSVC` with the default settings and we see it is able to predict the species of the flower fairly well.

In [15]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

svm_clf = Pipeline([
  ('scaler', StandardScaler()),
  ('svm', LinearSVC())
])
cross_val_score(svm_clf, X_train, y_train, cv=5)

array([0.95833333, 0.86956522, 0.95652174, 1.        , 0.95238095])

In [0]:
svm_clf.fit(X_train, y_train)
svm_clf.score(X_test, y_test)

0.9736842105263158

Now, we train an `SGDClassifier` on the same dataset.

In [0]:
from sklearn.linear_model import SGDClassifier

sgd_clf = Pipeline([
  ('scaler', StandardScaler()),
  ('sgd', SGDClassifier(alpha=0.01)),
])
cross_val_score(sgd_clf, X_train, y_train, cv=5)

array([0.79166667, 1.        , 0.95652174, 0.9047619 , 0.9047619 ])

In [0]:
sgd_clf.fit(X_train, y_train)
sgd_clf.score(X_test, y_test)

0.9736842105263158

Now we compare how each model would make predictions on the test set and we see that the models coincide exactly.

In [0]:
sgd_clf.score(X_test, svm_clf.predict(X_test))

1.0

### Exercise 9

In [0]:
# Some code to send me an email when the model finishes training.

import smtplib

def notify(msg):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("", "")
    server.sendmail(
        "",
        "", 
        'SUBJECT: Colab\n\n' + str(msg))
    server.quit()
    return msg

Fetching the MNIST dataset.

In [0]:
# Fetching the dataset.

from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8)
sorted_indices = np.argsort(mnist.target)
X = mnist.data[sorted_indices]
y = mnist.target[sorted_indices]

In [25]:
# Training a model using grid search

from sklearn.model_selection import GridSearchCV

rand_idx = np.random.permutation(len(X))

X_train, X_test, y_train, y_test = train_test_split(X[rand_idx[0:20]], y[rand_idx[0:20]])
param_grid = {
  'svm__kernel': ('linear', 'poly', 'rbf'),
  # 'svm__C': (10, 1, .1),
}
pipeline = Pipeline([
  ('scaler', StandardScaler()),
  ('svm', SVC()),
])
grid_search = GridSearchCV(pipeline, param_grid, cv=2).fit(X_train, y_train)
clf = grid_search.best_estimator_
notify('Best score: {:.4f}\n\nBest Params: {}\n\nBest Estimator: {}' + \
  '\n\nTest Score: {:.4f}'.format(
    grid_search.best_score_, grid_search.best_params_,
    clf, clf.score(X_test, y_test)))

SMTPAuthenticationError: ignored